In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [34]:
# Creating data frames for both csv files
genotypes_df = pd.read_csv(
    'data/genotypes.csv'
)
phenotypes_df = pd.read_csv(
    'data/phenotype_df.csv',
    usecols=[4]
)

In [35]:
genotypes_df

,sample_id,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_491,var_492,var_493,var_494,var_495,var_496,var_497,var_498,var_499,var_500
0,sample_0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,sample_1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,sample_2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,sample_3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,sample_4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,sample_1995,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1996,sample_1996,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1997,sample_1997,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1998,sample_1998,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
phenotypes_df

,LDL
0,0.628567
1,1.201150
2,0.497542
3,-1.705572
4,0.387260
...,...
1995,0.856525
1996,-0.534039
1997,0.430098
1998,-1.935098


In [89]:
# Excluding 'sample_id'
genotypes_vals = genotypes_df.iloc[:, 1:].values    

# Assuming equal weights
weights = np.ones(genotypes_df.shape[1]-1)

In [90]:
# Dot product for weighted sum
gene_burdens = np.dot(genotypes_vals, weights).reshape(1, -1)   # (1, n_samples)

# Adding B_0 term for predictor
X = np.concatenate(np.concatenate([np.ones((gene_burdens.shape[0], 1)), gene_burdens], axis=1))

# Target values
y = phenotypes_df.values

In [97]:
gene_burdens.shape

(1, 2000)

In [91]:
theta = np.linalg.inv(X.T@X) @ (X.T@y)

# Predicted values of y

yhat = X @ theta

plt.figure(1)
plt.plot(X,y,"bo", X, yhat, "r")


LinAlgError: 0-dimensional array given. Array must be at least two-dimensional